### Imports and Utility-Functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from numba import cuda

%run ./library/traffic_simulation.py
%run ../utils/helper.py

In [ ]:
def draw_lines_based_on_points_on_plot(points, ax=None, line_color='blue', line_size=0.5):
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 6))

    ax.plot(points[:, 0], points[:, 1], color=line_color, linewidth=line_size)

    return ax

def draw_points_on_plot(point_sequence, ax, line_color, point_size=1):
    x, y = zip(*point_sequence)
    ax.scatter(x, y, color=line_color, s=point_size)
    return ax

In [ ]:
def visualize_line_change_by_theta(line, theta):
    new_endpoint = calculate_rotated_line(line[0], line[1], theta)
    new_line = [line[1], new_endpoint]
    
    plt.plot([line[0][0], line[1][0]], [line[0][1], line[1][1]], 'r-', label='Original Line')

    # Plot the new line
    plt.plot([new_line[0][0], new_line[1][0]], [new_line[0][1], new_line[1][1]], 'b-', label='New Line')
    
    # Plotting points for clarity
    plt.scatter(*line[0], color='red', zorder=5)
    plt.scatter(*line[1], color='red', zorder=5)
    plt.scatter(*new_line[0], color='blue', zorder=5)
    plt.scatter(*new_line[1], color='blue', zorder=5)
    
    # Labeling the points
    plt.text(line[0][0], line[0][1], 'A', fontsize=12, ha='right')
    plt.text(line[1][0], line[1][1], 'B', fontsize=12, ha='right')
    plt.text(new_line[1][0], new_line[1][1], "B'", fontsize=12, ha='right')
    
    plt.xlim(-1, 2)
    plt.ylim(-1, 2)
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.axhline(0, color='black', linewidth=0.5)
    plt.axvline(0, color='black', linewidth=0.5)
    plt.grid(True)
    plt.legend()
    plt.title('Visualizing Line Transformation')
    plt.show()


In [ ]:
# a bit dirty. A good way to redraw
def redraw_initial_sketch():
    fig, ax = plt.subplots()
    ax.set_xlim(-30, 80)
    ax.set_ylim(-20, 70)
    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')
    
    angle_count_tuples = [[0, 60], [10, 9], [0, 20], [10, 9], [0, 60], [10, 9], [0, 20], [10, 9]]
    rotation_angles = calc_roation_angles_from_tuple(angle_count_tuples)
    init_point_sequence = [[0, 5], [1, 5]]
    point_sequence_one = generate_point_sequence(init_point_sequence, rotation_angles)
    ax = draw_lines_based_on_points_on_plot(point_sequence_one, ax, line_color='blue')
    
    angle_count_tuples = [[0, 60], [6, 15], [0, 20], [6, 15], [0, 60], [6, 15], [0, 20], [6, 15]]
    rotation_angles = calc_roation_angles_from_tuple(angle_count_tuples)
    init_point_sequence = [[-0.5, 1], [0.5, 1]]
    point_sequence_two = generate_point_sequence(init_point_sequence, rotation_angles)
    ax = draw_lines_based_on_points_on_plot(point_sequence_two, ax, line_color='red')

    return fig, ax

### Theory

In [ ]:
html(calculate_rotated_line)

In [ ]:
line = [[0, 0], [0.7071, 0.7071]]
theta = np.radians(0)
visualize_line_change_by_theta(line, theta)

In [ ]:
line = [[0, 0], [1, 0]]
theta = np.radians(10)
visualize_line_change_by_theta(line, theta)

### Streets-Building

In [ ]:
def calc_roation_angles_from_tuple(angle_count_tuples):
    return np.radians(np.repeat([angle for angle, count in angle_count_tuples], [count for angle, count in angle_count_tuples]))

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim(-30, 80)
ax.set_ylim(-20, 70)
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')

angle_count_tuples = [[0, 60], [10, 9], [0, 20], [10, 9], [0, 60], [10, 9], [0, 20], [10, 9]]
rotation_angles = calc_roation_angles_from_tuple(angle_count_tuples)
init_point_sequence = [[0, 5], [1, 5]]
point_sequence_one = generate_point_sequence(init_point_sequence, rotation_angles)
ax = draw_lines_based_on_points_on_plot(point_sequence_one, ax, line_color='blue')

angle_count_tuples = [[0, 60], [6, 15], [0, 20], [6, 15], [0, 60], [6, 15], [0, 20], [6, 15]]
rotation_angles = calc_roation_angles_from_tuple(angle_count_tuples)
init_point_sequence = [[-0.5, 1], [0.5, 1]]
point_sequence_two = generate_point_sequence(init_point_sequence, rotation_angles)
ax = draw_lines_based_on_points_on_plot(point_sequence_two, ax, line_color='red')

plt.show()

### Car-Simulation

In [ ]:
scatter_one = ax.scatter([], [], c='green', s=10)
scatter_two = ax.scatter([], [], c='orange', s=10)
time_scale = 5

def update(i):
    global ax, scatter_one, scatter_two, time_scale, point_sequence_one, point_sequence_two
    
    points_indices_one = np.mod(time_scale * np.array([2*i, i+20, i+50]), point_sequence_one.shape[0])
    points_indices_two = np.mod(time_scale * np.array([i+5, i+30, i+50]), point_sequence_two.shape[0])
    
    points_one = np.array([point_sequence_one[index] for index in points_indices_one])
    points_two = np.array([point_sequence_two[index] for index in points_indices_two])
    
    scatter_one.set_offsets(points_one)
    scatter_two.set_offsets(points_two)
    
    return scatter_one, scatter_two

ani = FuncAnimation(fig, update, frames=range(200), repeat=True, blit=True)
plt.close(fig)
HTML(ani.to_html5_video())

In [ ]:
N = 1000000
time_scale = 1
indices = np.arange(N)

indices_1 = np.mod(time_scale * np.array([2*indices, indices+20, indices+50]), point_sequence_one.shape[0])
indices_2 = np.mod(time_scale * np.array([indices+5, indices+30, indices+50]), point_sequence_two.shape[0])

points_one = point_sequence_one[indices_1]
points_two = point_sequence_two[indices_2]

#### Numpy-Operations

In [ ]:
def fill_zero_places(a):
    """
    a = np.array([0, 0, 0, 0,  3,  0,  0,  0,  0,  0, 11, 0, 0, 5, 0, 0, 17])
    b = np. array([ 0,  0,  0,  0,  0,  3,  3,  3,  3,  3,  3, 11, 11, 11,  5,  5,  5, 17])
    c = array([ 0,  0,  0,  0,  3,  3,  3,  3,  3,  3, 11, 11, 11,  5,  5,  5, 17])
    """
    a = np.append([-1], a)
    b = np.roll(a[a.nonzero()].repeat(np.diff(np.append(np.where(a!=0),a.shape[0]))), 0)
    b[b<0] = 0
    c = b[1:]
    return c

def shift_values_to_next_available_zero(a):
    """
    a = np.array([0, 2, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 6])
    b = np.array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1])
    c = np.array([0, 0, 2, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0,14, 0, 0, 0, 0, 0, 0])
    d = np.array([0, 2, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0,14, 0, 0, 0, 0, 0, 0])
    e = np.array([0, 0, 2, 2, 2, 2, 2, 2, 8, 8, 8, 8, 8, 8,14,14,14,14,14,14])
    f = np.array([2, 0, 6, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0])
    """
    """
    a = np.array([2, 2, 0, 0, 4, 0, 0, 6, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 6])
    f = np.array([2, 2, 4, 0, 0, 6, 0, 0, 4, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0]))
    """
    
    b = (a>0).astype(int)
    b = np.append([0], b)
    c = np.arange(b.shape[0]) * b
    c[-1] = -1 # acts as token
    c[-1] = 0
    d = c[1:]
    e = np.roll(fill_zero_places(d), 1)
    e[0] = 0
    f = np.zeros_like(a)
    np.add.at(f, e, a)
    f != 0
    
    return f

def find_zero_one_pattern(a):
    """
    a = np.array([0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1])
    b = np.array([1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
    """
    
    b = np.where((np.roll(a.astype(int), 1) == 1) & (a.astype(int) == 0), 1, 0)

    return b

def find_one_zero_pattern(a):
    """
    a = np.array([0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1])
    b = np.array([1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1])
    c = np.array([-1, 1, -1, 0, 0, 1, -1, 1, 0, 0, 0, 0, -1, 1, 0, 0, 0, 0, -1, 0]))
    d = np.array([0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])
    """

    # rules for a-b (a_i, b_i): (0, 1) -> -1, (0, 0) -> 0, (1, 0) -> 1, (1, 1) -> 0
    b = np.roll(a.astype(int), -1)
    b[-1] = 1 # to make the field invalid
    c = a-b
    d = ((c)==1).astype(int)
    
    return d

def fill_zero_values_ascending_rest_with_zero(a):
    """
    a = np.array([0, 2, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 6])
    b = np.array([0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1])
    c = np.array([1, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0])
    """
    
    b = (a>0).astype(int)
    c = (np.cumsum(1 - b) * (1-b) - np.maximum.accumulate(b * np.cumsum(1-b))) * ( 1 - b)

    return c

def find_values_that_must_be_reordered(a):
    """
    Each value of a must be at least as big as the index in the sequence: torch.min(a - torch.arange(0, a.shape[0])) >= 1
    a = array([11, 12, 23, 14, 14, 13, 60, 60])
    b = array([11, 12, 13, 13, 13, 13, 60, 60])
    c = array([0, 0, 1, 1, 1, 0, 1, 0])
    d = array([ 0,  0, 23, 14, 14,  0, 60,  0])
    e = array([0, 0, 1, 1, 1, 0, 1, 0])
    f = array([1, 1, 0, 0, 0, 1, 0, 1])
    g = array([0, 0, 3, 2, 1, 0, 1, 0]),
    h = array([ 0,  0, 20, 12, 13,  0, 59,  0])
    i = array([0, 0, 1, 1, 1, 0, 1, 0])
    """
    
    b = np.minimum.accumulate(a[::-1])[::-1]
    c = np.append((1 - (b[1:] - b[:-1]) > 0).astype(int), [0]) # places that might need change
    d = a * c
    e = (d >= b).astype(int) * c
    f = (1 - c)
    g = fill_zero_values_descending_rest_with_zero(f)
    h = d - g
    i = (h > b).astype(int) * c | (a >= b).astype(int) * c
    
    return i

def fill_zero_values_descending_rest_with_zero(a):
    """
    a = array([0, 2, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 6])
    b = array([1, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0])
    c = array([1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0])
    d = array([1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0])
    e = array([1, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 5, 0])
    f = array([1, 5, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0])
    g = array([1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
    h = array([1, 0, 5, 4, 3, 2, 1, 0, 5, 4, 3, 2, 1, 0, 5, 4, 3, 2, 1, 0])
    """
    
    b = fill_zero_values_ascending_rest_with_zero(a)
    c = (b>0).astype(int)
    d = find_one_zero_pattern(c)
    e = d * b
    f = shift_values_to_next_available_zero(e)
    g = fill_zero_places(f)
    h = ((g+1) - b) * c

    return h

def adjust_cars_no_lane_change(a):
    """
    a: array([10,  3,  4,  5,  7, 17, 15, 11]),
    b: array([ 1,  8,  7,  6,  4, -6, -4,  0]),
    m: array([1, 0, 0, 0, 0, 1, 1, 0]),
    c: array([0, 1, 0, 0, 0, 0, 0, 1]),
    d: array([1, 0, 1, 2, 3, 4, 5, 0]),
    e: array([ 0,  3,  0,  0,  0,  0,  0, 11]),
    f: array([ 3,  3,  3,  3,  3,  3, 11]),
    g: array([7, 0, 2, 2, 2, 2, 2, 2]),
    h: array([ 3,  0, 11,  0,  0,  0,  0,  0]),
    i: array([ 3,  3, 11, 11, 11, 11, 11, 11]),
    j: array([ 2,  3, 10,  9,  8,  7,  6, 11]),
    k: array([1, 0, 5, 4, 3, 2, 1, 0]),
    l: array([ 2,  3,  6,  7,  8,  9, 10, 11]),
    n: array([0, 1, 1, 1, 1, 0, 0, 1]),
    o: array([1, 0, 0, 0, 0, 2, 1, 0]),
    p: array([ 2,  0,  0,  0,  0,  9, 10,  0]),
    q: array([ 2,  3,  4,  5,  7,  9, 10, 11])
    """

    b = a[-1] - a
    m = find_values_that_must_be_reordered(b).astype(int)
    c = np.where((np.roll(m, 1) == 1) & (m == 0), 1, 0)
    d = (np.cumsum(1 - c) * (1 - c) - np.maximum.accumulate(c * np.cumsum(1 - c))) * (1 - c)
    e = a * c
    f = e[e.nonzero()].repeat(np.diff(np.append(np.where(e != 0), e.shape[0])))
    g = np.roll(np.maximum.accumulate(np.argsort(e) * c), 1)
    h = np.zeros_like(e)
    h[g] = e
    i = np.maximum.accumulate(h)
    j = i - d
    k = fill_zero_values_descending_rest_with_zero(c)
    l = i - k
    n = 1 - m
    o = fill_zero_values_descending_rest_with_zero(n) * m
    p = (i - o) * m
    q = p + n * a

    return q

#### Add cars to lanes, Respect other cars, Change lane

In [ ]:
def index_order_for_cars_in_lanes(car_specifications, lanes):
    lane_indices = np.array([car_specifications[car_specifications[:, 1] == i, 2].astype(int) for i in lanes])
    car_indicies = np.array([car_specifications[car_specifications[:, 1] == i, 0].astype(int) for i in lanes])
    return np.take_along_axis(car_indicies, np.argsort(lane_indices), axis=1)

In [ ]:
def update_lane_indicies(car_specifications, lane_num_tracks):
    car_specifications[:, 2] = np.mod(car_specifications[:, 3] + car_specifications[:, 2], lane_num_tracks[car_specifications[:, 1].astype(int)])
    return car_specifications

In [ ]:
def update_lane_indicies_with_overtake(car_specifications, lane_num_tracks, lanes):
    index_order = index_order_for_cars_in_lanes(car_specifications, lanes)
    car_specifications[:, 2] = np.mod(car_specifications[:, 3] + car_specifications[:, 2], lane_num_tracks[car_specifications[:, 1].astype(int)])
    updated_index_order = index_order_for_cars_in_lanes(car_specifications, lanes)

    
    # on collison, do not overtake
    
    return car_specifications

In [ ]:
# car_lane_zero = point_sequence_one
# car_lane_one = point_sequence_two

# lanes = np.array([0, 1])
# lane_num_tracks = np.array([point_sequence_one.shape[0], point_sequence_two.shape[0]])

# # car consists of id, driving_lane, point_index, speed and chance of swapping lane if waiting behind other car
# car_specifications = np.array([
#     [1, 0, 1, 3, 0.01], [2, 0, 20, 10, 0.01], [3, 0, 30, 3, 0.01], [4, 1, 5, 5, 0.01], [5, 1, 10, 7, 0.01], [6, 1, 25, 10, 0.01]
# ])

#### Speed-Up

In [ ]:
lane_num_tracks = np.array([198, 222])
cars_position = np.arange(0, 30)
cars_speed = np.random.randint(0,10, (30, 1)).flatten()

cars_position, cars_speed

#### Driving but no overtake

In [ ]:
# bug
a = np.array([ 83,  84,  89,  93,  87,  92,  90,  95,  90,  94,  94,  96,  97, 101,  97, 105, 104,  99, 101, 101, 101, 110, 105, 111, 106, 115, 90,  86, 107,  59])
adjust_cars_no_lane_change(a)

In [ ]:
cars_position = adjust_cars_no_lane_change(cars_position + cars_speed)
cars_position

In [ ]:
# fig, ax = redraw_initial_sketch()

# scatter_one = ax.scatter([], [], c='green', s=10)
# scatter_two = ax.scatter([], [], c='orange', s=10)

# def update(i):
#     global ax, scatter_one, scatter_two, point_sequence_one, point_sequence_two, car_specifications
    
#     points_one_indicies = car_specifications[car_specifications[:, 1].astype(int) == 0][:, 2].astype(int)
#     points_two_indicies = car_specifications[car_specifications[:, 1].astype(int) == 1][:, 2].astype(int)
    
#     points_one = point_sequence_one[points_one_indicies]
#     points_two = point_sequence_two[points_two_indicies]
    
#     scatter_one.set_offsets(points_one)
#     scatter_two.set_offsets(points_two)

#     lane_num_tracks = np.array([point_sequence_one.shape[0], point_sequence_two.shape[0]])
#     car_specifications = update_lane_indicies(car_specifications, lane_num_tracks)
#     # print(points_one)
    
#     return scatter_one, scatter_two

# ani = FuncAnimation(fig, update, frames=range(200), repeat=True, blit=True)
# plt.close(fig)
# HTML(ani.to_html5_video())

### Recording form fixed-car

### Recording from

### Replay from Recording